# AC-losses in an inductance with complex permeability

In [ ]:
# To install ngsolve
# !pip install ngsolve --upgrade
# !pip install webgui_jupyter_widgets --upgrade

import ngsolve as ngs
from ngsolve.webgui import Draw
from netgen.geom2d import CSG2d, Rectangle
from netgen.geom2d import EdgeInfo as EI, PointInfo as PI, Solid2d
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# Definition of the geometry

sl = 1e-2


def generate_mesh(air_gap=[1e-3] * 4, maxh=5e-3):
    h1, h2, h3, h4 = air_gap
    maxhP = maxh / 10
    h = 2 * sl
    lbox = 3 * h

    geo = CSG2d()
    box = Rectangle(
        pmin=(0, 0),
        pmax=(lbox, lbox),
        mat="air",
        left="dirichlet",
        right="dirichlet",
        top="dirichlet",
        bottom="neumann",
    )  # gray rectangle
    core = Solid2d(
        [
            (0, h1),
            PI(maxh=maxhP),
            EI(bc="airgap1"),
            (sl, h2),
            PI(maxh=maxhP),
            (sl, sl),
            PI(maxh=maxhP),
            (2 * sl, sl),
            PI(maxh=maxhP),
            (2 * sl, h3),
            PI(maxh=maxhP),
            EI(bc="airgap2"),
            (3 * sl, h4),
            PI(maxh=maxhP),
            (3 * sl, h),
            (0, h),
        ],
        mat="core",
    )
    coil = Rectangle(pmin=(sl, 0), pmax=(2 * sl, sl), mat="coil")
    geo.Add(box - core - coil)
    geo.Add(coil)
    geo.Add(core)
    mesh = ngs.Mesh(geo.GenerateMesh(maxh=maxh))
    return mesh


mesh = generate_mesh()

XiAir = mesh.MaterialCF({"air": 1})
XiCore = mesh.MaterialCF({"core": 1})
XiCoil = mesh.MaterialCF({"coil": 1})

Draw(XiCore + 2 * XiCoil, mesh)


In [ ]:
# Setup
nb_turn = 200
intensity = 1
omega = 2 * np.pi * 5e4

# Material properties
mu0 = 4e-7 * np.pi
mu_iron = 1000 * mu0
mu_coil = (0.9 - 0.1j) * mu0  #  AC losses in the copper from the imaginary part of the permeability
# mu_coil = (1.0 - 1e-16j) * mu0  #  Temporary

Jsource = intensity * nb_turn / ngs.Integrate(ngs.CF(1) * ngs.dx("coil"), mesh)


## 1) "Mixed" real formulation (workaround equivalent to complex)

How to deal with complex numbers ?

$$ \underline{A} \underline{x} = \underline{b} $$

Can be written only with real, using real and imaginary part and reads

$$ 
\begin{bmatrix}
\Re(\underline{A}) & -\Im(\underline{A}) \\
\Im(\underline{A}) & \Re(\underline{A})
\end{bmatrix}
\begin{bmatrix}
a_\Re\\
a_\Im
\end{bmatrix} = 
\begin{bmatrix}
\Re(\underline{b})\\
\Im(\underline{b})
\end{bmatrix} $$

The associated weak form is


$$  \left \{\begin{array}{lcl}
\displaystyle \int_\Omega \nabla a_\Re^* \cdot \nu_\Re \nabla a_\Re  -  \int_\Omega \nabla a_\Re^* \cdot \nu_\Im \nabla a_\Im  & = &  \displaystyle \int_\Omega a_\Re^* j \\
\displaystyle \int_\Omega \nabla a_\Im^* \cdot \nu_\Im \nabla a_\Re  +  \int_\Omega \nabla a_\Im^* \cdot \nu_\Re \nabla a_\Im  & = &  0
\end{array}
\right.$$

Since the objective function is 

$$ f = 4l\omega \int_{\Omega_c} \nu_{\Im} |\nabla a|^2  $$

the adjoint reads

$$ 
\begin{bmatrix}
\Re(\underline{A})^T & \Im(\underline{A})^T \\
-\Im(\underline{A})^T & \Re(\underline{A})^T
\end{bmatrix}
\begin{bmatrix}
p_\Re\\
p_\Im
\end{bmatrix} = 
\begin{bmatrix}
\partial_{a_\Re} f\\
\partial_{a_\Im}  f
\end{bmatrix} $$

$$  \left \{\begin{array}{lcl}
\displaystyle \int_\Omega \nabla a_\Re^* \cdot \nu_\Re \nabla a_\Re  +  \int_\Omega \nabla a_\Re^* \cdot \nu_\Im \nabla a_\Im  & = &  \displaystyle 4 l \omega  \int_\Omega \nabla a_\Re^* \cdot \nu_\Im  \nabla a_\Re \\
\displaystyle -\int_\Omega \nabla a_\Im^* \cdot \nu_\Im \nabla a_\Re  +  \int_\Omega \nabla a_\Im^* \cdot \nu_\Re \nabla a_\Im  & = &  4 l \omega  \int_\Omega \nabla a_\Im^* \cdot \nu_\Im  \nabla a_\Im
\end{array}
\right.$$

is equivalent to take the transpose-conjugate of the matrix (that is precisely the definition of the word "*adjoint*"!). Don't forget the conjugate :

$$ \underline{A} \underline{x} = \underline{b} $$
$$ \Rightarrow \underline{A}^* \underline{p} = \partial_{\underline{x}} f $$

Staying with real numbers implies that we can use the usual formulas to compute the sensitivities, which is nice. What is not nice: bigger and not symmetric system. Also, it would be simpler and more elegant to use only complex numbers.

In [ ]:
# Frequency domain "mixed" solver


def magWeakFormMixed(ar, ar_, ai, ai_):
    # real part
    bf = ngs.grad(ar_) * (1 / mu_iron).real * ngs.grad(ar) * ngs.dx("core")
    bf += -ngs.grad(ar_) * (1 / mu_iron).imag * ngs.grad(ai) * ngs.dx("core")
    bf += ngs.grad(ar_) * (1 / mu_coil).real * ngs.grad(ar) * ngs.dx("coil")
    bf += -ngs.grad(ar_) * (1 / mu_coil).imag * ngs.grad(ai) * ngs.dx("coil")
    bf += ngs.grad(ar_) * 1 / mu0 * ngs.grad(ar) * ngs.dx("air")

    # imag part
    bf += ngs.grad(ai_) * (1 / mu_iron).imag * ngs.grad(ar) * ngs.dx("core")
    bf += ngs.grad(ai_) * (1 / mu_iron).real * ngs.grad(ai) * ngs.dx("core")
    bf += ngs.grad(ai_) * (1 / mu_coil).imag * ngs.grad(ar) * ngs.dx("coil")
    bf += ngs.grad(ai_) * (1 / mu_coil).real * ngs.grad(ai) * ngs.dx("coil")
    bf += ngs.grad(ai_) * 1 / mu0 * ngs.grad(ai) * ngs.dx("air")

    lf = ar_ * Jsource * ngs.dx("coil") + ai_ * 0 * ngs.dx("coil")
    return bf, lf


def solveStateMixed(mesh, order=1):
    fesReal, fesImag = ngs.H1(mesh, order=order, dirichlet="dirichlet"), ngs.H1(
        mesh, order=order, dirichlet="dirichlet"
    )
    mixedSpace = fesReal * fesImag
    [ar, ai], [ar_, ai_] = mixedSpace.TnT()
    bf, f = magWeakFormMixed(ar, ar_, ai, ai_)
    K, F = ngs.BilinearForm(mixedSpace), ngs.LinearForm(mixedSpace)
    K += bf
    F += f
    K.Assemble()
    F.Assemble()
    gf = ngs.GridFunction(mixedSpace)
    Kinv = K.mat.Inverse(freedofs=mixedSpace.FreeDofs(), inverse="pardiso")  # not symmetric : don't use Cholesky !
    gf.vec.data = Kinv * F.vec
    return gf, Kinv


a0, Kinv = solveStateMixed(mesh)
Draw(a0.components[1])  # real part = 0, imag part = 1


In [ ]:
# Objective function (AC losses)

coeff_losses = 4 * sl * omega * (1 / mu_coil).imag
# coeff_losses = 4 * sl * w * (1 / muCoil).real  # Temporary


def localLossesMixed(a0):
    return coeff_losses * (ngs.grad(a0.components[0]) ** 2 + ngs.grad(a0.components[1]) ** 2) * ngs.dx("coil")


def objectiveMixed(a0, mesh):
    return ngs.Integrate(localLossesMixed(a0), mesh)


def dObjectiveMixed(ar, ar_, ai, ai_):
    return 2 * coeff_losses * (ngs.grad(ar) * ngs.grad(ar_) + ngs.grad(ai) * ngs.grad(ai_)) * ngs.dx("coil")


In [ ]:
# Adjoint solver
def solveAdjointMixed(a0, Kinv, df):
    fes = a0.space
    [pr, pi], [pr_, pi_] = fes.TnT()
    f = ngs.LinearForm(fes)
    f += df(a0.components[0], pr_, a0.components[1], pi_)
    f.Assemble()
    gf = ngs.GridFunction(fes)
    gf.vec.data = -1 * Kinv.T * f.vec
    return gf


p0 = solveAdjointMixed(a0, Kinv, dObjectiveMixed)
Draw(p0.components[1])  # real part = 0, imag part = 1
# Note that the imag part should is 0


In [ ]:
# Shape derivative and parametric sensitivity
def velocityFields(mesh):
    fes_velocity = ngs.VectorH1(mesh)
    v1, v2 = ngs.GridFunction(fes_velocity), ngs.GridFunction(fes_velocity)
    v3, v4 = ngs.GridFunction(fes_velocity), ngs.GridFunction(fes_velocity)
    v1.Set(ngs.CF((0, sl - ngs.x)) / sl, definedon=mesh.Boundaries("airgap1"))
    v2.Set(ngs.CF((0, ngs.x)) / sl, definedon=mesh.Boundaries("airgap1"))
    v3.Set(ngs.CF((0, 3 * sl - ngs.x)) / sl, definedon=mesh.Boundaries("airgap2"))
    v4.Set(ngs.CF((0, ngs.x - 2 * sl)) / sl, definedon=mesh.Boundaries("airgap2"))
    return v1, v2, v3, v4


def shapeDerivativeMixed(a0, p0, f):
    fes = a0.space
    bf, lf = magWeakFormMixed(a0.components[0], p0.components[0], a0.components[1], p0.components[1])
    Lagrangian = f(a0) + bf - lf

    # Computation of the shape derivative
    fesVec = ngs.VectorH1(mesh)
    dLOmega = ngs.LinearForm(fesVec)
    dLOmega += Lagrangian.DiffShape(fesVec.TestFunction())

    ## ... and assembly
    dLOmega.Assemble()
    dJ = ngs.GridFunction(fesVec)
    dJ.vec.FV().NumPy()[:] = dLOmega.vec.FV().NumPy()[:]
    return dJ


def gradientObjMixed(a0, p0):
    dJ_eCoil = shapeDerivativeMixed(a0, p0, localLossesMixed)
    g1 = ngs.InnerProduct(v1.vec, dJ_eCoil.vec)
    g2 = ngs.InnerProduct(v2.vec, dJ_eCoil.vec)
    g3 = ngs.InnerProduct(v3.vec, dJ_eCoil.vec)
    g4 = ngs.InnerProduct(v4.vec, dJ_eCoil.vec)
    return g1, g2, g3, g4


def computeLossShapeDerivativeMixed(a0, p0, f=None):
    fesVec = ngs.VectorH1(mesh)
    X = fesVec.TestFunction()
    ar = a0.components[0]
    pr = p0.components[0]
    ai = a0.components[1]
    pi = p0.components[1]

    rel = (1 / mu0) * XiAir + (1 / mu_coil) * XiCoil + (1 / mu_iron) * XiCore
    Id = ngs.CoefficientFunction((1, 0, 0, 1), dims=(2, 2))  # Identity matrix

    dLOmega = ngs.LinearForm(fesVec)

    dLOmega += ngs.SymbolicLFI(XiCoil * coeff_losses * ngs.InnerProduct(ngs.grad(ar), ngs.grad(ar)) * ngs.div(X))
    dLOmega += ngs.SymbolicLFI(XiCoil * coeff_losses * ngs.InnerProduct(ngs.grad(ai), ngs.grad(ai)) * ngs.div(X))

    dLOmega += ngs.SymbolicLFI(-XiCoil * 2 * coeff_losses * ngs.InnerProduct(ngs.grad(X) * ngs.grad(ar), ngs.grad(ar)))
    dLOmega += ngs.SymbolicLFI(-XiCoil * 2 * coeff_losses * ngs.InnerProduct(ngs.grad(X) * ngs.grad(ai), ngs.grad(ai)))

    dLOmega += ngs.SymbolicLFI(
        -Jsource * XiCoil * ngs.div(X) * pr
    )  # div(js X) = div(X)js + X * grad(js) = div(X)js a.e.

    dLOmega += ngs.SymbolicLFI(
        rel.real * ngs.InnerProduct((ngs.div(X) * Id - ngs.grad(X) - ngs.grad(X).trans) * ngs.grad(ar), ngs.grad(pr))
    )
    dLOmega += ngs.SymbolicLFI(
        -rel.imag * ngs.InnerProduct((ngs.div(X) * Id - ngs.grad(X) - ngs.grad(X).trans) * ngs.grad(ai), ngs.grad(pr))
    )
    dLOmega += ngs.SymbolicLFI(
        rel.real * ngs.InnerProduct((ngs.div(X) * Id - ngs.grad(X) - ngs.grad(X).trans) * ngs.grad(ai), ngs.grad(pi))
    )
    dLOmega += ngs.SymbolicLFI(
        rel.imag * ngs.InnerProduct((ngs.div(X) * Id - ngs.grad(X) - ngs.grad(X).trans) * ngs.grad(ar), ngs.grad(pi))
    )

    dLOmega.Assemble()
    dJ = ngs.GridFunction(fesVec)
    dJ.vec.FV().NumPy()[:] = dLOmega.vec.FV().NumPy()[:]
    return dJ


def gradientObjMixed_test(a0, p0):
    dJ_eCoil = computeLossShapeDerivativeMixed(a0, p0)
    g1 = ngs.InnerProduct(v1.vec, dJ_eCoil.vec)
    g2 = ngs.InnerProduct(v2.vec, dJ_eCoil.vec)
    g3 = ngs.InnerProduct(v3.vec, dJ_eCoil.vec)
    g4 = ngs.InnerProduct(v4.vec, dJ_eCoil.vec)
    return g1, g2, g3, g4


v1, v2, v3, v4 = velocityFields(mesh)
g1, g2, g3, g4 = gradientObjMixed(a0, p0)


In [ ]:
# Check with finite differences
a0, Kinv = solveStateMixed(mesh)
p0 = solveAdjointMixed(a0, Kinv, dObjectiveMixed)
v1, v2, v3, v4 = velocityFields(mesh)
g1, g2, g3, g4 = gradientObjMixed(a0, p0)
np.random.seed(0)
mesh = generate_mesh(air_gap=np.random.rand(4) * 5e-3, maxh=5e-3)
mu_iron = 1000 * mu0
a0, Kinv = solveStateMixed(mesh)
J0 = objectiveMixed(a0, mesh)
p0 = solveAdjointMixed(a0, Kinv, dObjectiveMixed)
v1, v2, v3, v4 = velocityFields(mesh)
g1, g2, g3, g4 = gradientObjMixed_test(a0, p0)
Eps = np.logspace(-8, -3, 10)
gradient_FD = []
taylorReminder = []
plt.figure()
for g, v in zip([g1, g2, g3, g4], [v1, v2, v3, v4]):
    gradient_FD.append([])
    taylorReminder.append([])
    for eps in Eps:
        V = ngs.GridFunction(v.space)
        V.Set(v * eps)
        mesh.SetDeformation(V)
        aTest, _ = solveStateMixed(mesh)
        JTest = objectiveMixed(aTest, mesh)
        gradient_FD[-1].append((JTest - J0) / eps)
        taylorReminder[-1].append(JTest - J0 - eps * g)
    plt.loglog(
        Eps,
        np.abs(taylorReminder[-1]),
        "o-",
        label="$R_{g" + str(len(taylorReminder)) + "}$",
    )

plt.title('Taylor test objective function ("mixed" formulation)')
plt.loglog(Eps, np.array(Eps) ** 2, "--", label="$ε^2$")
plt.xlabel("$ε$")
plt.ylabel("Taylor remainder")
plt.legend()
plt.show()
# Seems to work well !


## 2) Using fully complex formulation

Looking at the previous method, one can directly identify more compact complex formulations for the state and adjoint problems: $\forall a^* \in H^1_0(\Omega)$, find $\underline{a} \in H^1_0(\Omega,\mathbb{C})$:

$$\int_\Omega \nabla a^* \cdot \underline{\nu} \nabla \underline{a} = \int_\Omega a^* \underline{j}$$

with $\underline{j}$ actually real because we suppose the current density in phase with the reference (but it can have an imaginary part otherwise). Since the objective function is 

$$ f = 4l\omega \int_{\Omega_c} \nu_{\Im} |\nabla a|^2  $$

the adjoint reads

$$\int_\Omega \nabla \underline{p} \cdot \underline{\nu}^* \nabla a^* =2 \times 4 l \omega  \int_{\Omega_c} \nabla a^* \cdot \nu_\Im \nabla \underline{a} $$

with $\underline{\nu}^*$ the conjugate of $\underline{\nu}$. Since now the number are complex, one has to be careful with the derivation formulas, especially when applying the chain rule. 
Posing $\underline{a} = a_\Re + i a_\Im$, we extend the definition of the derivative :
- real w.r.t complex : $\partial_{\underline{a}} f = \partial_{a_\Re} f  + i \partial_{a_\Im} f$
- complex w.r.t real : $\partial_{x} \underline{a} = \partial_{x} a_\Re  + i \partial_x a_\Im$
- <span style="color:red">complex w.r.t complex</span>: not so simple (cf Wirtinger calculus) and I'm not so sure about that, but it is not explicitely needed in the final formula.

We can then use the following chain-rule:

$$ \mathrm{d}_x f = \partial_x f + \Re(\partial_{\underline{a}}f^* \cdot \mathrm{d}_x \underline{a} ) $$
$$ \Rightarrow \mathrm{d}_x f = \partial_x f + \Re(\underbrace{\partial_{\underline{a}}f^*\cdot \color{red}{\partial_{\underline{a}}  \underline{\mathcal R(\underline{a})}}^{-1}}_{\underline{p}^*} \cdot  \underbrace{\color{red}{\partial_{\underline{a}} \underline{\mathcal R(\underline{a})}} \cdot  \mathrm{d}_x \underline{a}}_{-\partial_x \underline{ \mathcal R(\underline{a})}} )$$
$$ \Rightarrow \boxed{\mathrm{d}_x f = \partial_x f - \Re(\underline{p}^* \cdot \partial_x \underline{ \mathcal R(\underline{a})})}$$

In [ ]:
# Frequency domain solver (complex)
mesh = generate_mesh()


def magWeakFormComplex(a, a_):
    bf = ngs.grad(a_) * 1 / mu_iron * ngs.grad(a) * ngs.dx("core")
    bf += ngs.grad(a_) * 1 / mu_coil * ngs.grad(a) * ngs.dx("coil")
    bf += ngs.grad(a_) * 1 / mu0 * ngs.grad(a) * ngs.dx("air")
    lf = a_ * Jsource * ngs.dx("coil")
    return bf, lf


# def magWeakFormComplex(a, a_):
#     bf = InnerProduct(grad(a_),  1 / muIron * grad(a)) * dx("core")
#     bf += InnerProduct(grad(a_),  1 / muCoil * grad(a))  * dx("coil")
#     bf += InnerProduct(grad(a_),  1 / mu0 * grad(a)) * dx("air")
#     lf = a_ * J * dx("coil")
#     return bf, lf


def solveStateComplex(mesh, order=1):
    complexSpace = ngs.H1(mesh, order=order, dirichlet="dirichlet", complex=True)
    a, a_ = complexSpace.TnT()
    bf, f = magWeakFormComplex(a, a_)
    K, F = ngs.BilinearForm(complexSpace), ngs.LinearForm(complexSpace)
    K += bf
    F += f
    K.Assemble()
    F.Assemble()
    gf = ngs.GridFunction(complexSpace)
    Kinv = K.mat.Inverse(freedofs=complexSpace.FreeDofs(), inverse="pardiso")  # Hermitian ?
    gf.vec.data = Kinv * F.vec
    return gf, Kinv


a0, Kinv = solveStateComplex(mesh)
Draw(a0.real, mesh)  # same as previously, as expected


In [ ]:
# Objective function (AC losses)


def localLossesComplex(a0):
    return coeff_losses * ngs.InnerProduct(ngs.grad(a0), ngs.grad(a0)) * ngs.dx("coil")
    # return coeffLosses * ((grad(a0).imag) ** 2 + (grad(a0).real) ** 2) * dx("coil")


def objectiveComplex(a0, mesh):
    return ngs.Integrate(localLossesComplex(a0), mesh)


def dObjectiveComplex(a, a_):
    return 2 * coeff_losses * (ngs.grad(a) * ngs.grad(a_)) * ngs.dx("coil")


def conj(a):
    return a.real - 1j * a.imag


In [ ]:
# Adjoint solver (complex)


def solveAdjointComplex(a0, Kinv, df):
    fes = a0.space
    p, p_ = fes.TnT()
    f = ngs.LinearForm(fes)
    f += df(a0, p_)
    f.Assemble()
    gf = ngs.GridFunction(fes)
    gf.vec.data = -1 * Kinv.H * f.vec
    return gf


p0 = solveAdjointComplex(a0, Kinv, dObjectiveComplex)
Draw(p0.real, mesh)  # same as previously, as expected
# Note that the imag part should is 0


In [ ]:
# Shape derivative and parametric sensitivity


def shapeDerivativeComplex(a0, p0, f):  # if there is a bug it should be in this function
    fes = a0.space
    bf, lf = magWeakFormComplex(a0, p0)
    Lagrangian = f(a0) + bf - lf

    # Computation of the shape derivative
    fesVec = ngs.VectorH1(mesh, complex=True)
    dLOmega = ngs.LinearForm(fesVec)
    dLOmega += Lagrangian.DiffShape(fesVec.TestFunction())  # here I'm not sure what is happening

    ## ... and assembly
    dLOmega.Assemble()
    dJ = ngs.GridFunction(ngs.VectorH1(mesh))
    dJ.vec.FV().NumPy()[:] = np.real(dLOmega.vec.FV().NumPy()[:])
    return dJ


def gradientObjComplex(a0, p0):
    dJ_eCoil = shapeDerivativeComplex(a0, p0, localLossesComplex)
    g1 = ngs.InnerProduct(v1.vec, dJ_eCoil.vec)
    g2 = ngs.InnerProduct(v2.vec, dJ_eCoil.vec)
    g3 = ngs.InnerProduct(v3.vec, dJ_eCoil.vec)
    g4 = ngs.InnerProduct(v4.vec, dJ_eCoil.vec)
    return g1, g2, g3, g4


v1, v2, v3, v4 = velocityFields(mesh)
g1, g2, g3, g4 = gradientObjComplex(a0, p0)


In [ ]:
# cf = ngs.CF(1j)
# cf2 = ngs.CF(2)

# Draw(ngs.InnerProduct(cf2, cf), mesh)


In [ ]:
def computeLossShapeDerivativeComplex(a0, p0, f=None):
    fesVec = ngs.VectorH1(mesh, complex=True)
    X = fesVec.TestFunction()

    rel = (1 / mu0) * XiAir + (1 / mu_coil) * XiCoil + (1 / mu_iron) * XiCore
    Id = ngs.CoefficientFunction((1, 0, 0, 1), dims=(2, 2))  # Identity matrix

    # dLOmega = ngs.LinearForm(fesVec)
    # dLOmega += ngs.SymbolicLFI(XiCoil * coeff_losses * ngs.InnerProduct((ngs.grad(a0)), ngs.grad(a0)) * ngs.div(X))
    # dLOmega += ngs.SymbolicLFI(-XiCoil * 2 * coeff_losses * ngs.InnerProduct(ngs.grad(X) * ngs.grad(a0), ngs.grad(a0)))
    # dLOmega += ngs.SymbolicLFI(
    #     -Jsource * XiCoil * ngs.div(X) * p0
    # )  # div(js X) = div(X)js + X * grad(js) = div(X)js a.e.
    # dLOmega += ngs.SymbolicLFI(
    #     ngs.InnerProduct(rel * (ngs.div(X) * Id - ngs.grad(X) - ngs.grad(X).trans) * ngs.grad(a0), ngs.grad(p0))
    # )

    dA = ngs.div(X) * Id - ngs.grad(X) - ngs.grad(X).trans
    dLOmega = ngs.LinearForm(fesVec)
    dLOmega += ngs.SymbolicLFI(XiCoil * coeff_losses * ngs.InnerProduct(dA * (ngs.grad(a0)), ngs.grad(a0)))
    dLOmega += ngs.SymbolicLFI(-Jsource * XiCoil * ngs.div(X) * p0)
    dLOmega += ngs.SymbolicLFI(ngs.InnerProduct(rel * dA * ngs.grad(a0), ngs.grad(p0)))

    dLOmega.Assemble()
    dJ = ngs.GridFunction(ngs.VectorH1(mesh))
    dJ.vec.FV().NumPy()[:] = np.real(dLOmega.vec.FV().NumPy()[:])
    return dJ


def gradientObjComplex_test(a0, p0):
    dJ_eCoil = computeLossShapeDerivativeComplex(a0, p0)
    g1 = ngs.InnerProduct(v1.vec, dJ_eCoil.vec)
    g2 = ngs.InnerProduct(v2.vec, dJ_eCoil.vec)
    g3 = ngs.InnerProduct(v3.vec, dJ_eCoil.vec)
    g4 = ngs.InnerProduct(v4.vec, dJ_eCoil.vec)
    return g1, g2, g3, g4


v1, v2, v3, v4 = velocityFields(mesh)
g1, g2, g3, g4 = gradientObjComplex_test(a0, p0)


In [ ]:
# Check with finite differences

np.random.seed(0)
mesh = generate_mesh(air_gap=np.random.rand(4) * 5e-3, maxh=5e-3)
mu_iron = 1000 * mu0
a0, Kinv = solveStateComplex(mesh)
J0 = objectiveComplex(a0, mesh)
p0 = solveAdjointComplex(a0, Kinv, dObjectiveComplex)
v1, v2, v3, v4 = velocityFields(mesh)
# g1, g2, g3, g4 = gradientObjComplex(a0, p0)
g1, g2, g3, g4 = gradientObjComplex_test(a0, p0)
Eps = np.logspace(-8, -3, 10)
gradient_FD = []
taylorReminder = []
plt.figure()
for g, v in zip([g1, g2, g3, g4], [v1, v2, v3, v4]):
    gradient_FD.append([])
    taylorReminder.append([])
    for eps in Eps:
        V = ngs.GridFunction(v.space)
        V.Set(v * eps)
        mesh.SetDeformation(V)
        aTest, _ = solveStateComplex(mesh)
        JTest = objectiveComplex(aTest, mesh)
        gradient_FD[-1].append((JTest - J0) / eps)
        taylorReminder[-1].append(JTest - J0 - eps * g)
    plt.loglog(
        Eps,
        np.abs(taylorReminder[-1]),
        "o-",
        label="$R_{g" + str(len(taylorReminder)) + "}$",
    )

plt.title("Taylor test objective function (complex formulation)")
plt.loglog(Eps, np.array(Eps) ** 2, "--", label="$ε^2$")
plt.xlabel("$ε$")
plt.ylabel("Taylor remainder")
plt.legend()
plt.show()


Suprisingly it works for the two geometric parameters located away from the coil, but not for the others.
Maybe `DiffShape` is not working totally for complex numbers, or maybe I did a mistake, probably in `shapeDerivativeComplex`.

In [ ]:
gradient_FD[0][0], gradient_FD[1][0], gradient_FD[2][0], gradient_FD[3][0]


In [ ]:
g1, g2, g3, g4


## Test optimization